In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 48.8 MB/s 
     |████████████████████████████████| 163 kB 61.7 MB/s 


In [ ]:
import transformers
import torch
import random
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast
import numpy as np
import os
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)



Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/943 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed(42)

In [ ]:
DEVICE = torch.device("cuda")
use_gpu = True
batch_size = 16

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
Dataset_all = pd.read_csv('drive/MyDrive/csv/phone.csv')

In [ ]:
Dataset_all

,id,text,label
0,0,3D Touch просто восхитительная вещь! Заряд дер...,1
1,1,"Отключается при температуре близкой к нулю, не...",0
2,2,"В Apple окончательно решили не заморачиваться,...",-1
3,3,Постарался наиболее ёмко и коротко описать все...,0
4,4,Достойный телефон. Пользоваться одно удовольст...,1
...,...,...,...
478046,478913,"часов разговоров, изредка пробки, инет очень ...",1
478047,478914,мой первый аппарат на андроиде. На данный моме...,1
478048,478915,"часа wi-fi, 1 час разговоров, 1 час радио/мр3...",1
478049,478916,Разбил iphone и не было желания покупать новый...,1


In [ ]:
Dataset = Dataset_all[200000:300000]

In [ ]:
Dataset

,id,text,label
200000,200706,Очень удачная модель. Идеал Андроида - звонилк...,1
200001,200707,При скорости выше 50км/ч будет проблематично з...,-1
200002,200708,"полгода телефон работал нормально,но потом пер...",-1
200003,200709,По-началу шрифт кажется непривычно мелким. Раз...,1
200004,200710,Вообще телефон на 5 с плюсом (не считая цены)....,1
...,...,...,...
299995,300758,"Первый телефон, более менее продвинутый, был Н...",1
299996,300759,"В целом, впечатления приятные. Немного омрачае...",1
299997,300760,"Я рекомендую этот телефон всем, кто захотел ег...",1
299998,300761,"Долго пользовался mi5s, когда презентовали mi6...",1


In [ ]:
import os
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [ ]:
Dataset = Dataset.dropna()

In [ ]:
Dataset.loc[(Dataset.label == -1), 'label'] = 2

In [ ]:
X = Dataset['text']

In [ ]:
y = Dataset['label']

In [ ]:
X

200000    Очень удачная модель. Идеал Андроида - звонилк...
200001    При скорости выше 50км/ч будет проблематично з...
200002    полгода телефон работал нормально,но потом пер...
200003    По-началу шрифт кажется непривычно мелким. Раз...
200004    Вообще телефон на 5 с плюсом (не считая цены)....
                                ...                        
299995    Первый телефон, более менее продвинутый, был Н...
299996    В целом, впечатления приятные. Немного омрачае...
299997    Я рекомендую этот телефон всем, кто захотел ег...
299998    Долго пользовался mi5s, когда презентовали mi6...
299999    Гб), и этот телефон не вызывает ощущения дешев...
Name: text, Length: 100000, dtype: object

In [ ]:
X.astype(str)

200000    Очень удачная модель. Идеал Андроида - звонилк...
200001    При скорости выше 50км/ч будет проблематично з...
200002    полгода телефон работал нормально,но потом пер...
200003    По-началу шрифт кажется непривычно мелким. Раз...
200004    Вообще телефон на 5 с плюсом (не считая цены)....
                                ...                        
299995    Первый телефон, более менее продвинутый, был Н...
299996    В целом, впечатления приятные. Немного омрачае...
299997    Я рекомендую этот телефон всем, кто захотел ег...
299998    Долго пользовался mi5s, когда презентовали mi6...
299999    Гб), и этот телефон не вызывает ощущения дешев...
Name: text, Length: 100000, dtype: object

In [ ]:
X_tokenized = tokenizer(X.astype(str).tolist(), max_length=512, padding=True, truncation=True, return_tensors='pt')

In [ ]:
X_tokenized

{'input_ids': tensor([[  101,  3065, 57050,  ...,     0,     0,     0],
        [  101,  1271,  4820,  ...,     0,     0,     0],
        [  101,  8977,  4484,  ...,     0,     0,     0],
        ...,
        [  101,   478, 10286,  ...,     0,     0,     0],
        [  101, 19409, 22825,  ...,     0,     0,     0],
        [  101, 48679,   122,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
X_tokenized['input_ids'][0]

tensor([  101,  3065, 57050,  6228,   132, 18717,  1444, 97022,   130, 22730,
          818,   132,  7959, 35459,  1088,   322, 28586,   353, 67582,   156,
          138,   132, 53420,   793,   322,  7991,   132,   140,   132, 49850,
        14859,   340, 57419,   322, 69856,   128,   322, 15154,  3128,   128,
          322,  1675, 26876,  1388,   156,   122,   142,   132, 26080,   906,
         6994, 84678,  1088,   156,   122,   144,   132, 78838,  1508,   128,
        65571,  1508,   128, 52795,   128, 73121,   128,  4484,   130,   888,
          918,  9528,   322,   340,  3895, 36947,   106,   146,   132,  3837,
         6992,   130, 14718, 16469,   802, 38793, 27872,   883, 23568,   156,
          122,  2630,   128,  1199,   840,   801, 14400, 18224, 72278,   130,
         1103,   322, 14718,  3397,   156,   122,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [ ]:
dataset = torch.utils.data.TensorDataset(X_tokenized['input_ids'], 
                                               X_tokenized['attention_mask'])

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=dataset, batch_size=batch_size)

In [ ]:
import torch.nn as nn

In [ ]:
model.to(DEVICE)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
def predict(model, train_loader):
    with torch.no_grad():
        logits = []
    
        for inputs, masks in train_loader:
            inputs = inputs.to(DEVICE)
            masks = masks.to(DEVICE)
            model.eval()
            outputs = model(input_ids=inputs, attention_mask=masks)[0].cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [ ]:
y_prob = predict(model, train_loader)

In [ ]:
y_prob

In [ ]:
y_pred = np.argmax(y_prob, axis=1)

In [ ]:
y_pred

In [ ]:
Dataset['model_predict'] = y_pred

In [ ]:
Dataset.to_csv("phone3_with_predict.csv", index = False)

In [ ]:
Dataset.to_csv('drive/MyDrive/csv/phone3_with_predict.csv', index = False)

In [ ]:
y_pred   

In [ ]:
#history = train(train_loader, val_loader, model=model.to(DEVICE), epochs=5, batch_size=batch_size)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np  

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
cnf_matrix = confusion_matrix(Dataset.label, Dataset.model_predict, labels=[0, 1, 2])
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, normalize=False, classes=['neutral', 'positive', 'negative'],
                      title='Confusion matrix, without normalization')